In [1]:
from config import OPENAI_API_KEY, TAVILY_API_KEY, LANGCHAIN_TRACING_V2, LANGCHAIN_API_KEY, ELEVEN_API_KEY

In [2]:
import json
from typing import Annotated, List, Sequence, TypedDict
from time import time
import operator
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langgraph.prebuilt import ToolInvocation, ToolExecutor
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
)
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import END, MessageGraph, StateGraph
from elevenlabs import play, stream, save
from elevenlabs.client import ElevenLabs

from IPython.display import Image

In [3]:
client = ElevenLabs()
llm = ChatOpenAI(model="gpt-4-turbo-2024-04-09", temperature=0)
tools = [TavilySearchResults(max_results=3)]
tool_executor = ToolExecutor(tools)
llm = llm.bind_tools(tools)

In [4]:
def text_to_speech(state):
    """
    Use this tools to transform text to speech
    """
    text = state["message"][-1].content
    audio_stream = client.generate(
        text=text,
        voice="Raimondo Marino",  # "Davide"
        model="eleven_multilingual_v2",
        stream=True,
    )
    return stream(audio_stream)

In [5]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # sender: str

In [6]:
# Define the function that determines whether to continue or not
def should_continue(state):
    messages = state["messages"]
    last_message = messages[-1]
    # If there is no function call, then we finish
    if not last_message.tool_calls:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

In [7]:
def call_model(state):
    messages = state["messages"]
    response = llm.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [8]:
def call_tool(state):
    messages = state["messages"]
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(
            last_message.additional_kwargs["function_call"]["arguments"]
        ),
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

In [9]:
def last_agent(state):
    human_input = state["messages"][-1].content
    return {
        "messages": [
            AIMessage(
                content="",
                tool_calls=[
                    {
                        "name": "text_to_speech",
                        "args": {
                            "text": human_input,
                        },
                        "id": "tool_abcd123",
                    }
                ],
            )
        ]
    }

In [11]:
builder = StateGraph(AgentState)

builder.add_node(key="agent", action=call_model)
builder.add_node(key="call_tool", action=call_tool)
builder.add_node(key="text_to_speech", action=text_to_speech)
builder.set_entry_point("agent")
builder.add_conditional_edges(
    start_key="call_tool",
    condition=should_continue,
    conditional_edge_mapping={
        "continue": "call_tool",
        "end": END,
    },
)
builder.add_edge("call_tool", "agent")
builder.set_finish_point("text_to_speech")

graph = builder.compile()

ValueError: Node `agent` is a dead-end